In [73]:
import re
import pandas as pd
import numpy as np

shark=pd.read_csv('attacks_new.csv')
shark=shark.loc[:,'Case Number':]
shark=shark.drop(['Case Number.1','Case Number.2','Unnamed: 22','Unnamed: 23'],axis=1)

In [74]:
pd.set_option('display.max_columns', 50)

In [75]:
act=r'([Ss]urf|[Ss]wim|[Bb]oat|[Ss]pear|[Ff]ish|[Bb]ath|[Ww]ading|[Dd]iving|[Ss]norkeling|[Bb]oard|[Ff]loat|[Rr]ow|[Ww]alking|[Kk]ayak|[Cc]anoe|[Ss]hip|[Ss]ki|[Ff]ell|[Ww]ash|[Tt]read|Unknown)'
shark['Activity_type']=shark['Activity'].str.extract(act)
shark['Activity_type']=shark['Activity_type'].fillna(value='other')

shark['Activity_type']=shark['Activity_type'].str.lower()

In [76]:
#read times into new column
shark['Hour'] = shark['Time'].str.extract(r'(\d+\D\d+)')
shark['Hour'] = shark['Hour'].str.replace(r'(\d+)\D(\d+)',r'\1\2')
shark['Hour'] = pd.to_numeric(shark['Hour'],errors='coerce')
shark['time_cat'] = pd.cut(shark['Hour'], [0,500,800,1100,1400,1700,2100,2400],labels=['Night','Early Morning','Morning','Midday','Afternoon','Evening','night'])
shark['time_cat'] = shark['time_cat'].str.lower()

pattern = r'([Aa]fternoon|[Mm]orning|[Nn]ight|[Ee]arly\s[Mm]orning|[Mm]idday|[Ee]vening)'
shark['time2'] = shark['Time'].str.extract(pattern)
shark['time_cat'] = shark['time_cat'].combine_first(shark['time2'].str.lower())

In [85]:
# with size -> group into ranges, convert meters to feet
# match digits
size_pattern_ft = r"([\d\.]+)(?:')"
size_pattern_m = r'([\d\.]+)(?: ?m)'

# extract based on patterns for feet and meters
shark['size_ft'] = shark['Species'].str.extract(size_pattern_ft)
shark['size_m'] = shark['Species'].str.extract(size_pattern_m)
# convert string to numeric
shark['size_ft'] = pd.to_numeric(shark['size_ft'])
shark['size_m'] = pd.to_numeric(shark['size_m'],errors='coerce') #multiply to convert meters to feet

#combine_first prioritizes first column so meter sizes are only used where ft is NaN
shark['size'] = shark.size_ft.combine_first(shark.size_m*3.3)
# group sizes into categories
shark['size_cat'] = pd.cut(shark['size'], [0,5,10,15,25,400],labels=['0-5','5-10','10-15','15-25','25+'])

# remove 25+ entry because it is incorrectly reading millimeters
shark[shark['size_cat'] == '25+'] = np.nan

shark['size_cat'].value_counts()

5-10     858
0-5      569
10-15    346
15-25    171
25+        0
Name: size_cat, dtype: int64

In [80]:
# shark species can have 1 or 2 names
# pattern selects a word with 3 or more letters and a group with a word followed by shark
species_pattern = r'([a-zA-Z]{3,})?\s*([a-zA-Z]{3,}\s*shark)'
# extract based on the pattern into a dataframe with two columns
spec_df = shark['Species'].str.extract(species_pattern)
# some species only have one word in addition to shark
spec_col = spec_df[1]
# if there is a match for the first group, concatenate the columns together
spec_col[spec_df[0].isnull() == False] = spec_df[0].str.cat(spec_df[1], sep='')
spec_col = spec_col.str.lower()
# Consolidate names for same shark species
spec_col[spec_col == 'zambesi shark'] = 'tiger shark'
# remove small shark because it is not a species
spec_col[spec_col == 'small shark'] = np.nan

shark['shark_species'] = spec_col

In [81]:
shark.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,Full Location,Latitude,Longitude,Activity_type,Hour,time_cat,time2,size_ft,size_m,size,size_cat,shark_species
0,2018.06.25,25-Jun-18,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6303,"Oceanside, San Diego County, California, USA",33.195870,-117.379483,other,1800.0,evening,NaN,NaN,NaN,NaN,NaN,white shark
1,2018.06.18,18-Jun-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ...,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6302,"St. Simon Island, Glynn County, Georgia, USA",NaN,NaN,other,1400.0,midday,NaN,NaN,NaN,NaN,NaN,NaN
2,2018.06.09,9-Jun-18,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6301,"Habush, Oahu, Hawaii, USA",NaN,NaN,surf,745.0,early morning,NaN,NaN,NaN,NaN,NaN,NaN
3,2018.06.08,8-Jun-18,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6300,"Arrawarra Headland, New South Wales, AUSTRALIA",-30.060426,153.201685,surf,NaN,NaN,NaN,NaN,2.0,6.6,5-10,NaN
4,2018.06.04,4-Jun-18,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6299,"La Ticla, Colima, MEXICO",NaN,NaN,diving,NaN,NaN,NaN,NaN,3.0,9.9,5-10,tiger shark


In [84]:
shark['size_cat'].value_counts()

5-10     858
0-5      569
10-15    346
15-25    171
25+        1
Name: size_cat, dtype: int64